Check Tide Map
==============

Check if a given point is within a tide model domain

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  
- https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/
- ftp://ftp.esr.org/pub/datasets/tmd/  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  

Finite Element Solution (FES) provided by AVISO  
- https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html

#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  
 - [ipyleaflet: Jupyter / Leaflet bridge enabling interactive maps](https://github.com/jupyter-widgets/ipyleaflet)  

#### Program Dependencies

- `convert_ll_xy.py`: convert lat/lon points to and from projected coordinates  
- `read_tide_model.py`: extract tidal harmonic constants from OTIS tide models  
- `read_netcdf_model.py`: extract tidal harmonic constants from netcdf models  
- `read_GOT_model.py`: extract tidal harmonic constants from GSFC GOT models  
- `read_FES_model.py`: extract tidal harmonic constants from FES tide models  

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
The widgets can be installed as described below.  
```
pip3 install --user ipywidgets
jupyter nbextension install --user --py widgetsnbextension
jupyter nbextension enable --user --py widgetsnbextension
jupyter-notebook
```

#### Load modules

In [ ]:
from __future__ import print_function

import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipyleaflet as leaflet

import pyTMD.time
import pyTMD.read_tide_model
import pyTMD.read_netcdf_model
import pyTMD.read_GOT_model
import pyTMD.read_FES_model
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# set the directory with tide models
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)

# dropdown menu for setting tide model
model_list = ['CATS0201','CATS2008','TPXO9-atlas','TPXO9-atlas-v2',
    'TPXO9-atlas-v3','TPXO9-atlas-v4','TPXO9.1','TPXO8-atlas','TPXO7.2',
    'AODTM-5','AOTIM-5','AOTIM-5-2018',
    'GOT4.7','GOT4.8','GOT4.10','FES2014']
modelDropdown = widgets.Dropdown(
    options=model_list,
    value='GOT4.10',
    description='Model:',
    disabled=False,
)

# display widgets for setting directory, model and date
widgets.VBox([dirText,modelDropdown])

In [ ]:
#-- default coordinates to use
LAT,LON = (32.93301304,242.7294513)
m = leaflet.Map(center=(LAT,LON), zoom=12, basemap=leaflet.basemaps.Esri.WorldTopoMap)
#-- add control for zoom
zoom_slider = widgets.IntSlider(description='Zoom level:', min=0, max=15, value=7)
widgets.jslink((zoom_slider, 'value'), (m, 'zoom'))
zoom_control =  leaflet.WidgetControl(widget=zoom_slider, position='topright')
m.add_control(zoom_control)
#-- add marker with default location
marker = leaflet.Marker(location=(LAT,LON), draggable=True)
m.add_layer(marker)
#-- add text with marker location
markerText = widgets.Text(
    value='{0:0.8f},{1:0.8f}'.format(LAT,LON),
    description='Lat/Lon:',
    disabled=False
)

#-- add function for setting marker text if location changed
def set_marker_text(sender):
    LAT,LON = marker.location
    markerText.value = '{0:0.8f},{1:0.8f}'.format(LAT,LON % 360)
#-- add function for setting map center if location changed
def set_map_center(sender):
    m.center = marker.location
#-- add function for setting marker location if text changed
def set_marker_location(sender):
    LAT,LON = [float(i) for i in markerText.value.split(',')]
    marker.location = (LAT,LON % 360)
    
#-- watch marker widgets for changes
marker.observe(set_marker_text)
markerText.observe(set_marker_location)
m.observe(set_map_center)
#-- add control for marker location
marker_control =  leaflet.WidgetControl(widget=markerText, position='bottomright')
m.add_control(marker_control)
m

In [ ]:
# directory with tide models
tide_dir = os.path.expanduser(dirText.value)
TIDE_MODEL = modelDropdown.value
LAT,LON = marker.location
# verify longitudes
LON %= 360

# select between tide models
if (TIDE_MODEL == 'CATS0201'):
    grid_file = os.path.join(tide_dir,'cats0201_tmd','grid_CATS')
    reference = 'https://mail.esr.org/polar_tide_models/Model_CATS0201.html'
    model_format = 'OTIS'
    EPSG = '4326'
elif (TIDE_MODEL == 'CATS2008'):
    grid_file = os.path.join(tide_dir,'CATS2008','grid_CATS2008')
    model_format = 'OTIS'
    EPSG = 'CATS2008'
elif (TIDE_MODEL == 'TPXO9-atlas'):
    grid_file = os.path.join(tide_dir,'TPXO9_atlas','grid_tpxo9_atlas.nc.gz')
    model_format = 'netcdf'
elif (TIDE_MODEL == 'TPXO9-atlas-v2'):
    grid_file = os.path.join(tide_dir,'TPXO9_atlas_v2','grid_tpxo9_atlas_30_v2.nc.gz')
    model_format = 'netcdf'
elif (TIDE_MODEL == 'TPXO9-atlas-v3'):
    grid_file = os.path.join(tide_dir,'TPXO9_atlas_v3','grid_tpxo9_atlas_30_v3.nc.gz')
    model_format = 'netcdf'
elif (TIDE_MODEL == 'TPXO9-atlas-v4'):
    grid_file = os.path.join(tide_dir,'TPXO9_atlas_v4','grid_tpxo9_atlas_30_v4')
    model_format = 'OTIS'
    EPSG = '4326'
    TYPE = 'z'
elif (TIDE_MODEL == 'TPXO9.1'):
    grid_file = os.path.join(tide_dir,'TPXO9.1','DATA','grid_tpxo9')
    model_format = 'OTIS'
    EPSG = '4326'
elif (TIDE_MODEL == 'TPXO8-atlas'):
    grid_file = os.path.join(tide_dir,'tpxo8_atlas','grid_tpxo8atlas_30_v1')
    model_format = 'ATLAS'
    EPSG = '4326'
elif (TIDE_MODEL == 'TPXO7.2'):
    grid_file = os.path.join(tide_dir,'TPXO7.2_tmd','grid_tpxo7.2')
    model_format = 'OTIS'
    EPSG = '4326'
elif (TIDE_MODEL == 'AODTM-5'):
    grid_file = os.path.join(tide_dir,'aodtm5_tmd','grid_Arc5km')
    model_format = 'OTIS'
    EPSG = 'PSNorth'
elif (TIDE_MODEL == 'AOTIM-5'):
    grid_file = os.path.join(tide_dir,'aotim5_tmd','grid_Arc5km')
    model_file = os.path.join(tide_dir,'aotim5_tmd','h_Arc5km.oce')
    model_format = 'OTIS'
    EPSG = 'PSNorth'
elif (TIDE_MODEL == 'AOTIM-5-2018'):
    grid_file = os.path.join(tide_dir,'Arc5km2018','grid_Arc5km2018')
    model_file = os.path.join(tide_dir,'Arc5km2018','h_Arc5km2018')
    model_format = 'OTIS'
    EPSG = 'PSNorth'
elif (TIDE_MODEL == 'GOT4.7'):
    model_directory = os.path.join(tide_dir,'GOT4.7','grids_oceantide')
    model_files = ['q1.d.gz','o1.d.gz','p1.d.gz','k1.d.gz','n2.d.gz',
        'm2.d.gz','s2.d.gz','k2.d.gz','s1.d.gz','m4.d.gz']
    model_format = 'GOT'
    SCALE = 1.0/100.0
elif (TIDE_MODEL == 'GOT4.8'):
    model_directory = os.path.join(tide_dir,'got4.8','grids_oceantide')
    model_files = ['q1.d.gz','o1.d.gz','p1.d.gz','k1.d.gz','n2.d.gz',
        'm2.d.gz','s2.d.gz','k2.d.gz','s1.d.gz','m4.d.gz']
    model_format = 'GOT'
    SCALE = 1.0/100.0
elif (TIDE_MODEL == 'GOT4.10'):
    model_directory = os.path.join(tide_dir,'GOT4.10c','grids_oceantide')
    model_files = ['q1.d.gz','o1.d.gz','p1.d.gz','k1.d.gz','n2.d.gz',
        'm2.d.gz','s2.d.gz','k2.d.gz','s1.d.gz','m4.d.gz']
    model_format = 'GOT'
    SCALE = 1.0/100.0
elif (TIDE_MODEL == 'FES2014'):
    model_directory = os.path.join(tide_dir,'fes2014','ocean_tide')
    model_files = ['2n2.nc.gz','eps2.nc.gz','j1.nc.gz','k1.nc.gz',
        'k2.nc.gz','l2.nc.gz','la2.nc.gz','m2.nc.gz','m3.nc.gz','m4.nc.gz',
        'm6.nc.gz','m8.nc.gz','mf.nc.gz','mks2.nc.gz','mm.nc.gz',
        'mn4.nc.gz','ms4.nc.gz','msf.nc.gz','msqm.nc.gz','mtm.nc.gz',
        'mu2.nc.gz','n2.nc.gz','n4.nc.gz','nu2.nc.gz','o1.nc.gz','p1.nc.gz',
        'q1.nc.gz','r2.nc.gz','s1.nc.gz','s2.nc.gz','s4.nc.gz','sa.nc.gz',
        'ssa.nc.gz','t2.nc.gz']
    model_format = 'FES'
    TYPE = 'z'
    SCALE = 1.0/100.0

# adjust dimensions of input coordinates to be iterable
LON = np.atleast_1d(LON)
LAT = np.atleast_1d(LAT)
    
# read tidal constants and interpolate to grid points
if model_format in ('OTIS','ATLAS'):
    # if reading a single OTIS solution
    xi,yi,hz,mz,iob,dt = pyTMD.read_tide_model.read_tide_grid(grid_file)
    # adjust dimensions of input coordinates to be iterable
    # run wrapper function to convert coordinate systems of input lat/lon
    x,y = pyTMD.convert_ll_xy(np.atleast_1d(LON),np.atleast_1d(LAT),EPSG,'F')
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    if (np.min(x) < np.min(xi)) & (EPSG == '4326'):
        lt0, = np.nonzero(x < 0)
        x[lt0] += 360.0
    if (np.max(x) > np.max(xi)) & (EPSG == '4326'):
        gt180, = np.nonzero(x > 180)
        x[gt180] -= 360.0
elif (model_format == 'netcdf'):
    # if reading a netCDF OTIS atlas solution
    xi,yi,hz = pyTMD.read_netcdf_model.read_netcdf_grid(grid_file,
        GZIP=True, TYPE='z')
    # invert bathymetry mask
    mz = np.invert(hz.mask)
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    x,y = np.copy([LON,LAT]).astype(np.float64)
    lt0, = np.nonzero(x < 0)
    x[lt0] += 360.0
elif (model_format == 'GOT'):
    # if reading a NASA GOT solution
    input_file = os.path.join(model_directory,model_files[0])
    hc,xi,yi,c = pyTMD.read_GOT_model.read_GOT_grid(input_file, GZIP=True)
    # invert tidal constituent mask
    mz = np.invert(hc.mask)
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    x,y = np.copy([LON,LAT]).astype(np.float64)
    lt0, = np.nonzero(x < 0)
    x[lt0] += 360.0
elif (model_format == 'FES'):
    # if reading a FES netCDF solution
    input_file = os.path.join(model_directory,model_files[0])
    hc,xi,yi = pyTMD.read_FES_model.read_netcdf_file(input_file,GZIP=True,
        TYPE='z',VERSION='FES2014')
    # invert tidal constituent mask
    mz = np.invert(hc.mask)
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    x,y = np.copy([LON,LAT]).astype(np.float64)
    lt0, = np.nonzero(x < 0)
    x[lt0] += 360.0

# check coordinates on tide grid
fig,ax = plt.subplots(num=1,figsize=(12,12), dpi=200)
ax.imshow(mz, interpolation='nearest',
    extent=(xi.min(),xi.max(),yi.min(),yi.max()),
    origin='lower', cmap='gray')
ax.plot(x,y,'r*')
# no ticks on the x and y axes
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()